<a href="https://colab.research.google.com/github/FIJY/hh/blob/master/hh2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests bs4 lxml fake_useragent pandas


In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import time


In [3]:
# Мы создаем сессию requests, которая позволит нам использовать один и тот же заголовок User-Agent для всех запросов. Это также улучшит производительность, поскольку сессия может переиспользовать соединения.

session = requests.Session()
ua = UserAgent()
session.headers['User-Agent'] = ua.random


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from fake_useragent import UserAgent

session = requests.Session()
ua = UserAgent()
session.headers['User-Agent'] = ua.random

def get_links(industry_code, age_from, age_to):
    base_url = "https://hh.ru/search/resume"
    params = {
        #"exp_period": "all_time",
        "filter_exp_industry": industry_code,
        "logic": "normal",
        "no_magic": "true",
        "order_by": "publication_time",
        "ored_clusters": "true",
        #"pos": "full_text",
        "search_period": "30",
        "items_on_page": "50",
        #"filter_exp_period": "all_time",
        "relocation": "living_or_relocation",
        "gender": "unknown",
        "text": "",
        "label": "only_with_age",
        "age_from": str(age_from),
        "age_to": str(age_to) if age_to is not None else ""
    }

    query_url = f"{base_url}?{'&'.join(f'{k}={v}' for k, v in params.items() if v is not None)}"
    response = session.get(query_url)
    soup = BeautifulSoup(response.content, 'lxml')
    header = soup.find('h1', {'data-qa': 'bloko-header-3'})

    if header:
        total_resumes = header.get_text()
        total_resumes = int(re.search(r'\d+', total_resumes.replace('\xa0', '')).group(0))
    else:
        total_resumes = 0  # Assume no resumes found if header is missing

    return total_resumes, query_url

industry_code = "7.540"
age_ranges_results = []
age_from = 0
max_resumes = 4999

while age_from < 100:
    age_to = age_from
    total_resumes = 0

    # Expand age range until resumes exceed the maximum limit or the age range covers the remaining ages
    while total_resumes <= max_resumes and age_to < 100:
        age_to += 1
        total_resumes, query_url = get_links(industry_code, age_from, age_to)
        if total_resumes > max_resumes:
            age_to -= 1  # Step back to the last valid group
            total_resumes, query_url = get_links(industry_code, age_from, age_to)
            break

    # Append the results
    age_ranges_results.append((age_from, age_to, total_resumes, query_url))
    print(f"Возрастная группа {age_from}-{age_to}: {total_resumes} резюме найдено. Проверить: {query_url}")
    age_from = age_to + 1  # Move to the next start age beyond the current range

# Write the results to a CSV file
with open('age_group_links.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Age From', 'Age To', 'Total Resumes', 'URL'])
    for age_from, age_to, total_resumes, url in age_ranges_results:
        writer.writerow([age_from, age_to, total_resumes, url])

print("Ссылки и данные сохранены в файл 'age_group_links2.csv'")



Возрастная группа 0-19: 2616 резюме найдено. Проверить: https://hh.ru/search/resume?filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&search_period=30&items_on_page=50&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=0&age_to=19
Возрастная группа 20-20: 2730 резюме найдено. Проверить: https://hh.ru/search/resume?filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&search_period=30&items_on_page=50&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=20&age_to=20
Возрастная группа 21-21: 4156 резюме найдено. Проверить: https://hh.ru/search/resume?filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&search_period=30&items_on_page=50&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=21&age_to=21
Возрастная группа 22-22: 5251 резюме найдено. Проверить: https://h

In [5]:
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time

def parse_resumes(base_url):
    ua = fake_useragent.UserAgent()
    headers = {"User-Agent": ua.random}

    # Получаем количество страниц для обхода
    response = requests.get(f"{base_url}&page=1", headers=headers)
    if response.status_code != 200:
        print(f"Ошибка доступа к странице: {response.status_code}")
        return

    soup = BeautifulSoup(response.content, 'lxml')
    try:
        page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span", recursive=False)[-1].find("a").find("span").text)
    except Exception as e:
        print(f"Ошибка при получении количества страниц: {e}")
        page_count = 1  # Если не удалось определить количество, пробуем хотя бы одну страницу

    # Обход каждой страницы для парсинга данных
    for page in range(1, page_count + 1):
        response = requests.get(f"{base_url}&page={page}", headers=headers)
        if response.status_code != 200:
            print(f"Ошибка при доступе к странице {page}: {response.status_code}")
            continue

        soup = BeautifulSoup(response.content, 'lxml')
        links = [f"https://hh.ru{a.attrs['href'].split('?')[0]}" for a in soup.find_all("a", attrs={"data-qa": "serp-item__title"})]
        print(f"Ссылки на странице {page}:")
        for link in links:
            print(link)
        time.sleep(1)  # Задержка между запросами для избежания блокировки

# Пример использования
base_url = input("Введите базовую ссылку для парсинга резюме: ")
parse_resumes(base_url)


Введите базовую ссылку для парсинга резюме: https://hh.ru/search/resume?exp_period=all_time&filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&pos=full_text&search_period=30&items_on_page=50&filter_exp_period=all_time&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=0&age_to=19
Ошибка при получении количества страниц: 'NoneType' object has no attribute 'find_all'
Ссылки на странице 1:



    # Собираем базовую информацию
    resume = {
        "gender": get_detailed_info(soup, "resume-personal-gender"),
        "age": get_detailed_info(soup, "resume-personal-age"),
        "city": get_detailed_info(soup, "resume-personal-address"),
        "desired_salary": get_detailed_info(soup, "resume-block__salary"),
        "relocation": get_detailed_info(soup, "resume-personal-relocation"),
        "specializations": get_detailed_info(soup, "resume-block-specialization", "class"),
        "employment": get_detailed_info(soup, "resume-block-employment", "data-qa"),
        "work_schedule": get_detailed_info(soup, "resume-block-schedule", "data-qa"),
        "experience": get_detailed_info(soup, "resume-block-experience"),
        "key_skills": [tag.text for tag in soup.find_all(attrs={"class": "bloko-tag__section_text"})],
        "about_me": get_detailed_info(soup, "resume-block-skills-content"),
        "languages": get_detailed_info(soup, "resume-block-languages"),
        "citizenship": get_detailed_info(soup, "resume-personal-citizenship"),
        "education": get_detailed_info(soup, "resume-block-education-content"),
        "link": link
    }

In [20]:
import requests
from bs4 import BeautifulSoup
import fake_useragent
import random
import time

def parse_resumes(base_url, login, password, filename):
    session = requests.Session()
    ua = fake_useragent.UserAgent()
    session.headers.update({"User-Agent": ua.random})

    login_url = "https://hh.ru/account/login"
    login_data = {'username': login, 'password': password}
    session.post(login_url, data=login_data)

    with open(filename, 'w') as file:
        response = session.get(f"{base_url}&page=1")
        if response.status_code != 200:
            print(f"Ошибка доступа к странице: {response.status_code}")
            return

        soup = BeautifulSoup(response.content, 'lxml')
        try:
            page_count = int(soup.find("div", attrs={"class": "pager"}).find_all("span", recursive=False)[-1].find("a").find("span").text)
        except Exception as e:
            print(f"Не удалось определить количество страниц, используем одну страницу для парсинга. Ошибка: {e}")
            page_count = 1

        for page in range(1, page_count + 1):
            response = session.get(f"{base_url}&page={page}")
            if response.status_code != 200:
                print(f"Ошибка при доступе к странице {page}: {response.status_code}")
                continue

            soup = BeautifulSoup(response.content, 'lxml')
            links = [f"https://hh.ru{a.attrs['href'].split('?')[0]}" for a in soup.find_all("a", attrs={"data-qa": "serp-item__title"})]
            print(f"Обрабатывается страница {page}, найдено {len(links)} ссылок")
            for link in links:
                file.write(link + '\n')
            time.sleep(random.uniform(3, 8))  # Рандомная задержка между запросами

        print(f"Ссылки были сохранены в файл {filename}")

# Используйте значения по умолчанию для упрощения вызова функции
login = "sivilio@sivilio.by"
password = "Sivilio4UII"
base_url = "https://hh.ru/search/resume?exp_period=all_time&filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&pos=full_text&search_period=30&items_on_page=50&filter_exp_period=all_time&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=0&age_to=19"
filename = "/content/it_age_0_19_v3"

parse_resumes(base_url, login, password, filename)


Обрабатывается страница 1, найдено 20 ссылок
Обрабатывается страница 2, найдено 20 ссылок


KeyboardInterrupt: 

In [22]:
import requests
from bs4 import BeautifulSoup
import fake_useragent
import random
import time

def parse_resumes(base_url, login, password, filename):
    session = requests.Session()
    ua = fake_useragent.UserAgent()
    session.headers.update({"User-Agent": ua.random})

    login_url = "https://hh.ru/account/login"
    login_data = {'username': login, 'password': password}
    session.post(login_url, data=login_data)

    with open(filename, 'w') as file:
        response = session.get(f"{base_url}&page=1")
        if response.status_code != 200:
            print(f"Ошибка доступа к странице: {response.status_code}")
            return

        soup = BeautifulSoup(response.content, 'lxml')
        # Логирование для диагностики
        with open("debug.html", "w") as debug_file:
            debug_file.write(response.text)

        try:
            page_count = int(soup.find("div", attrs={"class": "pager"}).find_all("span", recursive=False)[-1].find("a").find("span").text)
        except Exception as e:
            print(f"Не удалось определить количество страниц, используем одну страницу для парсинга. Ошибка: {e}")
            page_count = 1

        for page in range(1, page_count + 1):
            response = session.get(f"{base_url}&page={page}")
            soup = BeautifulSoup(response.content, 'lxml')
            links = [f"https://hh.ru{a.attrs['href'].split('?')[0]}" for a in soup.find_all("a", attrs={"data-qa": "serp-item__title"})]
            print(f"Обрабатывается страница {page}, найдено {len(links)} ссылок")
            for link in links:
                file.write(link + '\n')
            time.sleep(random.uniform(1, 5))  # Рандомная задержка между запросами

        print(f"Ссылки были сохранены в файл {filename}")

login = "sivilio@sivilio.by"
password = "Sivilio4UII"
base_url = "https://hh.ru/search/resume?exp_period=all_time&filter_exp_industry=7.540&logic=normal&no_magic=true&order_by=publication_time&ored_clusters=true&pos=full_text&search_period=30&items_on_page=20&filter_exp_period=all_time&relocation=living_or_relocation&gender=unknown&text=&label=only_with_age&age_from=0&age_to=19"
filename = "/content/it_age_0_19_v4"

parse_resumes(base_url, login, password, filename)


Обрабатывается страница 1, найдено 19 ссылок
Обрабатывается страница 2, найдено 20 ссылок
Обрабатывается страница 3, найдено 20 ссылок
Обрабатывается страница 4, найдено 20 ссылок
Обрабатывается страница 5, найдено 20 ссылок
Обрабатывается страница 6, найдено 20 ссылок
Обрабатывается страница 7, найдено 20 ссылок
Обрабатывается страница 8, найдено 20 ссылок
Обрабатывается страница 9, найдено 20 ссылок
Обрабатывается страница 10, найдено 20 ссылок
Обрабатывается страница 11, найдено 20 ссылок
Обрабатывается страница 12, найдено 20 ссылок
Обрабатывается страница 13, найдено 20 ссылок
Обрабатывается страница 14, найдено 20 ссылок
Обрабатывается страница 15, найдено 20 ссылок
Обрабатывается страница 16, найдено 20 ссылок
Обрабатывается страница 17, найдено 20 ссылок
Обрабатывается страница 18, найдено 20 ссылок
Обрабатывается страница 19, найдено 20 ссылок
Обрабатывается страница 20, найдено 20 ссылок
Обрабатывается страница 21, найдено 20 ссылок
Обрабатывается страница 22, найдено 20 ссыл

In [23]:
def count_lines(filename):
    with open(filename, 'r') as file:
        return sum(1 for line in file)

filename = "/content/it_age_0_19_v4"
print(f"Количество строк в файле: {count_lines(filename)}")


Количество строк в файле: 2143
